<a href="https://colab.research.google.com/github/jmcaussade/Homologacion-Car-Makers/blob/main/NHTSA_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import requests
import time

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files

In [4]:
ruta_data_set = '/content/drive/MyDrive/Xinergy/Proyecto_homologacion_autos/database.xlsx'
df = pd.read_excel(ruta_data_set, sheet_name='Anexo 6')
df.head()
df

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,car_type,Peso kg
0,NaN,355845,Auto,2023-09-01 00:01:13,SZD861A,3HGCM66563G001615,2003.0,estandar,Honda,ACCORD,Auto,NaN
1,NaN,355847,Camion,2023-09-01 00:06:09,TP9498G,3N6DD25T2EK005444,2014.0,estandar,NISSAN,ESTACAS,camion,NaN
2,NaN,355849,Auto,2023-09-01 00:08:38,NCZ1892,MEX5G2603JT120368,2018.0,estandar,Volkswagen,VENTO,Auto,NaN
3,NaN,355857,Auto,2023-09-01 00:24:34,FCJ597B,1C4BJWEG6FL629655,2015.0,estandar,JEEP,J WRANGLER,Auto,NaN
4,NaN,355864,Auto,2023-09-01 00:46:09,NJA2747,MA6CB6CD6KT029772,2019.0,estandar,CHEVROLET,BEAT,Auto,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
279424,NaN,745808,Equipo Ligero,2024-08-31 00:00:00,TDY040B,3N1CN7AD7KK399688,2019.0,NaN,NISSAN,VERSA,Equipo Ligero,NaN
279425,NaN,745810,Equipo Ligero,2024-08-31 00:00:00,JUB4510,LSJA24U99RN010717,2024.0,NaN,MG,HS,Equipo Ligero,NaN
279426,NaN,745811,Equipo Ligero,2024-08-31 00:00:00,RTR920A,3G1SF21X97S112886,2007.0,NaN,CHEVROLET,CHEVY,Equipo Ligero,NaN
279427,NaN,745814,Equipo Ligero,2024-08-31 00:00:00,PWEE63E,LJD5AA194R0124080,2024.0,NaN,KIA,SELTOS,Equipo Ligero,NaN


In [7]:
df = df.sample(n=1000, random_state=42)  # Randomly selects 1,000 rows


In [8]:
df

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,car_type,Peso kg,clean_car_brand
108468,NaN,555333,Auto,2024-03-03 12:14:38,JSR9842,3G1TA5AF2EL201086,2014.0,estandar,CHEVROLET,AVEO,Auto,NaN,chevrolet
266747,NaN,668826,Equipo Ligero,2024-06-18 00:00:00,FZY023C,3N1CK3CD6LL238145,2020.0,NaN,NISSAN,MARCH,Equipo Ligero,NaN,nissan
211939,NaN,739778,Auto,2024-08-26 12:07:24,S/N,3VWS1A1B91M900418,2001.0,estandar,ESPECIALES,AUTOMOVILES,Auto,NaN,especiales
140294,NaN,613969,Auto,2024-04-28 13:42:51,HNY531D,9BHCR4AC7PP437009,2023.0,estandar,HYUNDAI,HB20,Auto,NaN,hyundai
103149,NaN,545535,Auto,2024-02-22 21:30:22,PPK458B,JHLRD78716C402853,2006.0,estandar,Honda,CR-V,Auto,NaN,honda
...,...,...,...,...,...,...,...,...,...,...,...,...,...
243807,NaN,529779,Equipo Ligero,2024-02-08 00:00:00,RZS146B,5N1AR2MN4GC629938,2016.0,NaN,NISSAN,PATHFINDER,Equipo Ligero,NaN,nissan
212923,NaN,741542,Auto,2024-08-27 21:23:59,S/N,LSJA36E96PZ039041,2023.0,estandar,MG,5,Auto,NaN,mg
255439,NaN,600551,Equipo Ligero,2024-04-16 00:00:00,SPS542B,988675432PKM46574,2023.0,NaN,JEEP,COMPASS,Equipo Ligero,NaN,jeep
82537,NaN,506741,Auto,2024-01-16 15:43:37,UCW204A,1N4AL3AP6FN332051,2015.0,estandar,NISSAN,ALTIMA,Auto,NaN,nissan


In [9]:
df["clean_serie"] = df["serie"].fillna("").astype(str).str.strip()


In [10]:
df["Maker_api"] = None  # Add a new column for results

In [11]:
# API Function
def get_maker(vin):
    url = f"https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVin/{vin}?format=json"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            results = response.json().get("Results", [])
            for item in results:
                if item["Variable"] == "Make":
                    return item["Value"]
        return None
    except Exception as e:
        print(f"Error for VIN {vin}: {e}")
        return None

In [12]:
for index, row in df.iterrows():
    vin = row["clean_serie"]
    maker = get_maker(vin)
    df.at[index, "Maker_api"] = maker
    print(f"Processed VIN {index}: {vin} -> {maker}")

    # Add a delay to respect API rate limits
    time.sleep(0.1)

Processed VIN 108468: 3G1TA5AF2EL201086 -> CHEVROLET
Processed VIN 266747: 3N1CK3CD6LL238145 -> None
Processed VIN 211939: 3VWS1A1B91M900418 -> VOLKSWAGEN
Processed VIN 140294: 9BHCR4AC7PP437009 -> None
Processed VIN 103149: JHLRD78716C402853 -> HONDA
Processed VIN 155840: VWASHTF2XJ1206769 -> None
Processed VIN 41984: 5NPD74AF3KH490916 -> HYUNDAI
Processed VIN 232055: JS2ZC13S4K6101288 -> SUZUKI
Processed VIN 115036: 3N6AD35C7JK893790 -> NISSAN
Processed VIN 30546: KL8CM6DAXKC764729 -> CHEVROLET
Processed VIN 216931: 3N1CN7AD8HL833653 -> NISSAN
Processed VIN 73202: 2BLB1318089 -> None
Processed VIN 140310: 1HGCR2657HA900645 -> HONDA
Processed VIN 152633: 3N8CP5HD0JL486895 -> NISSAN
Processed VIN 4422: 5N1AR18U99C603060 -> NISSAN
Processed VIN 193345: VR3EF9HP5LJ500159 -> None
Processed VIN 81879: 3G1MA5E26ML132218 -> CHEVROLET
Processed VIN 186867: 2GNFLEEKXD6117002 -> CHEVROLET
Processed VIN 193267: 93CCL8008CB107444 -> None
Processed VIN 207311: 3N1AB61D68L704725 -> NISSAN
Processed

In [13]:
df.to_excel("sample_test_results_NHTSA.xlsx", index=False)
print("Sample test results saved to sample_test_results.xlsx")


Sample test results saved to sample_test_results.xlsx
